In [ ]:
# imports

import chromadb
from chromadb.utils import embedding_functions

import json
import chromadb
from chromadb.config import Settings
from chromadb import EmbeddingFunction
from chromadb.utils import embedding_functions

In [ ]:
## Terminal command to start a local host server
## chroma run --path "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data"

## instantiate chroma client
# chroma_client = chromadb.HttpClient(host="localhost", port=8000)

chroma_internet_client = chromadb.HttpClient(host='16.171.68.145', port=8000)

In [ ]:
# Function to create multiple collections
persistence_database_path_windows = "G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/chroma_data"

persistence_database_path_mac = "/Users/vasu/Desktop/NLP /project/chatbot_ic_NLP/lib/backend/data/chroma_data"

persistence_database_path_ubuntu = "/home/ubuntu/chromadb"

chroma_internet_client = chromadb.PersistentClient(path=persistence_database_path_ubuntu, settings=Settings(allow_reset=True))

def create_chroma_collections(years):
    
    sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)
    
    for year in years:
        # Specify the path to your JSON file
        json_file_path = f"G:/All Flutter Applications/NLP with transformers project/chatbot_ic/lib/backend/data/data_json/{year}pubmed.json"

        # Open the JSON file and load the data
        with open(json_file_path, 'r') as json_file:
            dataset = json.load(json_file)
            
        collection = chroma_internet_client.create_collection(f"{year}pubmed", embedding_function=sentence_transformer_ef, metadata={"hnsw:space": "cosine"})
        
        collection.add(
            ids=[str(entry['PMID']) for entry in dataset],
            documents= [ entry['PMID'] + "<SEP>" +  entry['Author']+ "<SEP>" + entry['Title'] + "<SEP>" + entry['Abstract'] for entry in dataset],
            metadatas=[
                {'author': entry['Author']} for entry in dataset
            ],
        )

In [ ]:
# chroma_internet_client.reset()

In [ ]:
years = ['2013', '2014', '2015', '2016-2017', '2018', '2019', '2020-1', '2020-2']  

create_chroma_collections(years=years)

In [ ]:
## access 2013 collection
collection_2013 = chroma_internet_client.get_collection('2013pubmed')

In [ ]:
from sentence_transformers import SentenceTransformer, util

sentence_transformer_ef = embedding_functions.SentenceTransformerEmbeddingFunction(model_name="multi-qa-MiniLM-L6-cos-v1", normalize_embeddings=True)

# User's question
user_question = 'CASK Disorder'

# Embed the user's question
user_question_embedding = sentence_transformer_ef([user_question])[0]

search_results = collection_2013.query(query_embeddings=[user_question_embedding], n_results=5)
print(user_question)
# Print the search results
for i, result in enumerate(search_results['ids'][0]):
    document_id = result
    metadata = search_results['metadatas'][0][i]  # Access the corresponding metadata
    similarity_score = search_results['distances'][0][i]  # Access the corresponding similarity score
    document = search_results['documents'][0][i]  # Access the corresponding document

    print(f"PMID: {document_id}")
    # print(f"Title: {metadata['title']}")
    print(f"Author: {metadata['author']}")
    print(document)
    print(f"Similarity Score: {similarity_score}")
    print("---------------")


In [ ]:
chroma_internet_client.list_collections()